# CIFAR-10 Image Classification with CNN

## Objective
Build and train a Convolutional Neural Network (CNN) from scratch using TensorFlow and Keras to classify images from the CIFAR-10 dataset.
This project demonstrates data loading, preprocessing, model architecture design, data augmentation, training with callbacks, and comprehensive evaluation.

## Requirements
- Setup Python environment with TensorFlow, NumPy, Matplotlib, Seaborn, Scikit-learn.
- Achieve >75% accuracy on test set.
- Implement Data Augmentation and Learning Rate Scheduling.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow Version: {tf.__version__}")

## 1. Data Loading and Preprocessing
We load the CIFAR-10 dataset, which consists of 60,000 32x32 color images in 10 classes.
The pixel values are normalized to the range [0, 1].

In [ ]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Define class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print(f"Training data shape: {x_train.shape}")
print(f"Test data shape: {x_test.shape}")

### Exploratory Data Analysis (EDA)
Visualizing sample images from the dataset.

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    # The CIFAR labels happen to be arrays, hence the extra index
    plt.xlabel(class_names[y_train[i][0]])
plt.show()

## 2. CNN Architecture
We design a Sequential CNN model with:
- 3 Convolutional blocks (Conv2D + ReLU + MaxPool + Dropout)
- Flatten layer
- Dense layers with Dropout
- Final Softmax layer for 10 classes

This architecture is designed to capture hierarchical features while preventing overfitting using Dropout.

In [ ]:
def create_model():
    model = Sequential([
        Input(shape=(32, 32, 3)),
        
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
        
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

model = create_model()
model.summary()

## 3. Data Augmentation
To improve generalization, we use `ImageDataGenerator` to augment the training data with random horizontal flips, rotations, and shifts.

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(x_train)

## 4. Training
We compile the model with Adam optimizer and Categorical Crossentropy loss.
We use callbacks:
- `ReduceLROnPlateau`: To lower learning rate if validation loss plateaus.
- `ModelCheckpoint`: To save the best model.

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint('cifar10_best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("GPU detected. Setting epochs to 50 for full training.")
    epochs = 50
else:
    print("No GPU detected. Training on CPU is very slow.")
    print("Setting epochs to 5 for demonstration purposes. Accuracy will be low.")
    print("To achieve >75% accuracy, please run this on a machine with a GPU (e.g., Google Colab) for 50 epochs.")
    epochs = 5

batch_size = 64

history = model.fit(datagen.flow(x_train, y_train_cat, batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test_cat),
                    callbacks=[reduce_lr, checkpoint, early_stop],
                    verbose=1)

## 5. Evaluation
We evaluate the model on the test set and visualize the performance.
Note: If you are running this notebook, the following cells depend on the trained model.

In [ ]:
# Load best model if available, else use current
try:
    best_model = load_model('cifar10_best_model.keras')
    print("Loaded best model from disk")
except:
    print("Using current model (checkpoints not found)")
    best_model = model

loss, acc = best_model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

### Classification Report and Confusion Matrix

In [ ]:
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

print("Classification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

### Misclassified Images
Visualizing 10 images where the model made errors.

In [ ]:
misclassified_idx = np.where(y_pred_classes != y_true)[0]
plt.figure(figsize=(15, 6))
if len(misclassified_idx) > 0:
    for i in range(min(10, len(misclassified_idx))):
        idx = misclassified_idx[i]
        plt.subplot(2, 5, i+1)
        plt.imshow(x_test[idx])
        plt.title(f"True: {class_names[y_true[idx]]}\nPred: {class_names[y_pred_classes[idx]]}", color='red')
        plt.axis('off')
    plt.show()

### Feature Maps Visualization
Visualizing the output of the first convolutional layer for a sample input.

In [ ]:
# Extract output of the first Conv2D layer
import numpy as np
from tensorflow.keras.models import Model

# Ensure model is built (Keras 3 fix)
try:
    best_model.predict(np.zeros((1, 32, 32, 3)), verbose=0)
except:
    pass

layer_outputs = [layer.output for layer in best_model.layers if 'conv2d' in layer.name]
activation_model = Model(inputs=best_model.inputs, outputs=layer_outputs)

sample_image = x_test[0].reshape(1, 32, 32, 3)
activations = activation_model.predict(sample_image)

first_layer_activation = activations[0]
print(f"First Conv2D Activation Shape: {first_layer_activation.shape}")

plt.figure(figsize=(16, 8))
num_filters = first_layer_activation.shape[-1]
cols = 8
rows = num_filters // cols

for i in range(min(32, num_filters)):
    plt.subplot(rows if rows > 0 else 1, cols, i+1)
    plt.imshow(first_layer_activation[0, :, :, i], cmap='viridis')
    plt.axis('off')
plt.suptitle('Feature Maps (First Conv2D Layer)')
plt.show()